In [44]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

In [23]:
chromedriver_location = "chrome_driver/chromedriver"

In [24]:
district_drop_down = '//*[@id="courtId"]'
date_field = '//*[@id="inputVO.startDate"]'
seven_day_search_radio = '//*[@id="sevenDayRadio"]'
search_by_attorney_radio = '//*[@id="searchAttorneyRadio"]'
mobar_input = '//*[@id="inputVO.mobarNumber"]'

submit_button = '//*[@id="findButton"]'

In [45]:
driver = webdriver.Chrome(chromedriver_location)
driver.get('https://www.courts.mo.gov/casenet/cases/calendarSearch.do')

courtDistrictDrp = Select(driver.find_element_by_xpath(district_drop_down))
courtDistrictDrp.select_by_value("CT11")

driver.find_element_by_xpath(date_field).send_keys("8/9/2021")
driver.find_element_by_xpath(seven_day_search_radio).click()
driver.find_element_by_xpath(search_by_attorney_radio).click()
driver.find_element_by_xpath(mobar_input).send_keys("71855")

time.sleep(1)

driver.find_element_by_xpath(submit_button).click()

